##### DNN 기반 회귀 모델 구현
- 데이터 : iris.csv
- 피처/속성 : 3개 Sepal.Length, Sepal.Width, Petal.Length
- 타겟/라벨 : 1개 Petal.Width
- 학습-방법 : 지도학습 > 회귀
- 학습 알고리즘 : 인공신경망(ANN) -> 심층 신경망 (MLP, DNN) : 은닉층이 많은 구성
- 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
# 모델 관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 및 시각화 관련 모듈
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기~~~
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [3]:
### 데이터 로딩
DATA_FILE = '../../ML/Data/iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매 개 변 수 : 층별 입출력 개수 고정하므로 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : __init__() : 모델 구조 설정, forward() : 순방향 학습 <=오버라이딩(상속관계에서만 가능)
- 클래스 구조
    * 입력층 : 입력 3개(피처 개수) / 출력 10개(퍼셉트론/뉴런 개수 10개)
    * 은닉층 : 입력 10개          / 출력 30개
    * 출력층 : 입력 30개          / 출력 1개(너비값)
- - -
- 손실함수 / 활성화 함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
    * 함수 형태 ==> torch.nn.fuctional 아래에 ==> forward() 메서드

In [4]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer = nn.Linear(3,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)

    #순방향 학습 진행 메서드
    def forward(self,x) : 
        # 입력층
        y = self.in_layer(x)    # 
        y=F.relu(y)             # relu 값의 범위 : 0<=y / 시그모이드 : 0~1
        # 은닉층 : 10개 숫자의 값(>=0)
        y = self.hidden_layer(y)
        y = F.relu(y)
        # 출력층 : 30개 숫자값 / 회귀이므로 바로 반환
        return self.out_layer(y)

In [5]:
# 모델 인스턴스 생성
model = IrisRegModel()
print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [6]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(1500,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1500, 1]                 --
├─Linear: 1-1                            [1500, 10]                40
├─Linear: 1-2                            [1500, 30]                330
├─Linear: 1-3                            [1500, 1]                 31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.60
Input size (MB): 0.02
Forward/backward pass size (MB): 0.49
Params size (MB): 0.00
Estimated Total Size (MB): 0.51

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피처 개수 : 3개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 / 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * _ _len_ _(self) : 데이터의 개수 반환
    * _ _getItem_ _(self, index) : 특정 인덱스의 피처와 타겟 반환

In [7]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) : 
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        feaureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피처와 타겟 반환
        return feaureTS, targetTS

In [8]:
# 피처와 타겟 데이터 추출
featureDF = irisDF[irisDF.columns[:-1]]     # 2D (150,3)
targetDF = irisDF[irisDF.columns[-1:]]      # 2D (150,1)

In [9]:
## 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비
- 학습 횟수 : EPOCH   <- 처음부터 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE   <- 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE  <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학 습 율  : LR 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001~0.1 (하이퍼파라미터)

In [10]:
### 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [11]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train,X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')

# irisDS = IrisDataset(X_train, y_train)
trainDS = IrisDataset(X_train, y_train)
valDS = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test, y_test)

# 데이터로더 인스턴스
# irisDL = DataLoader(irisDS, batch_size = BATCH_SIZE)

trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)


In [12]:
## [테스트] 데이터 로더
for feature, target in trainDS:
    print(feature.shape, target.shape)

torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Size([3]) torch.Size([1])
torch.Si

In [ ]:
val_feaure_TS = torch.FloatTensor(valDS.featureDF.values)
val_target_TS = torch.FloatTensor(valDS.targetDF.values)

In [13]:
# 최적화 인스턴스 => W,b 텐서, 즉 model.parameters() 전달 - 최적화하는 이유 : 오차를 줄이기 위해서!
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀 : MSE, MAE, RMSE
reqLoss = nn.MSELoss()

[5] 학습 진행

In [16]:
## 학습의 효과 확인 - 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = irisDS.n_rows/BATCH_SIZE
print(f'CNT =>{CNT}')

for epoch in range(EPOCH):
    # 학습 모드로 모델 성정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0
    for featureTS, targetTS in trainDL :
        # 학습 진행
        pre_y = model(featureTS)

        # 손실계산
        loss = reqLoss(pre_y, targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score = R2Score()(pre_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad():
        # 검증용 데이터셋 생성
        val_feaure_TS = torch.FloatTensor(valDS.featureDF.values)
        val_target_TS = torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val = model(val_feaure_TS)
        # 손실 계산
        loss_val = reqLoss(pre_val, val_target_TS)
        # 성능 평가
        score_val = R2Score()(pre_val, val_target_TS)

    # 에포크 당 손실과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

CNT =>15.0
